# 1. Data preprocessing - frequency statistics in arguments

Analyzing the frequency of words in claim and premise. The purpose is to find a useful feature in classifying claim and premise. 

## input the labeled sents data

In [1]:
import pickle 

In [2]:
with open('labeled_essay_dics.pickle', 'rb') as handle:
    label_sents = pickle.load(handle)

In [3]:
# compute and store and the {sents: label} into one dic
all_label_sents = {}
for key in label_sents.keys():
    this_essay = label_sents[key]
    for s in this_essay.keys():
        all_label_sents[s] = this_essay[s]

In [4]:
all_sents = ""
for key in all_label_sents.keys():
    all_sents = all_sents + " " + key

In [5]:
premise_sents = ""
major_claim_sents = ""
claim_sents = ""
all_claim_sents = ""
other_arg_sents = ""

In [6]:
for key in all_label_sents.keys():
    this_value = all_label_sents[key]
    if this_value == "Premise":
        premise_sents = premise_sents + " " + key
    elif this_value == "Claim":
        claim_sents = claim_sents + " " + key
        all_claim_sents = all_claim_sents + " " + key
    elif this_value == "MajorClaim":
        major_claim_sents = major_claim_sents + " " + key
        all_claim_sents = all_claim_sents + " " + key
    else:
        other_arg_sents = other_arg_sents + " " + key 

## sents cleaning

In [7]:
import re
import itertools
from collections import Counter

In [8]:
"""
Original taken from https://github.com/dennybritz/cnn-text-classification-tf
"""


def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()

In [9]:
def clear_split_str(s):
    strip_s = s.strip()
    clear_s = clean_str(strip_s)
    s_text = clear_s.split(" ")
    return s_text

In [10]:
clear_all_sents = clear_split_str(all_sents)
clear_premise_sents = clear_split_str(premise_sents)
clear_major_claim_sents = clear_split_str(major_claim_sents)
clear_claim_sents = clear_split_str(claim_sents)
clear_all_claim_sents = clear_split_str(all_claim_sents)
clear_other_arg_sents = clear_split_str(other_arg_sents)

## compute frequency in claims and premises 


In [11]:
from nltk import FreqDist

In [12]:
# check the word that appear in "claim" and "premise" at same time
i = 0
for key in  all_label_sents.keys():
    
    if "recognize" in clear_split_str(key) and all_label_sents[key] == 'Premise':
        print (key)
        print (i)
        i = i + 1 

In other words, once you are special and unique, pursuing success is a matter of actively creating the fashion rather than passively waiting for other people to recognize and appreciate you.
0
It is clear to recognize that the internet service is being provided at a low cost or even free in many countries.
1


In [13]:
# FreqDist(clear_premise_sents).most_common(200)

In [14]:
# 

# visualize the result

In [15]:
'''
The code is from https://stackoverflow.com/questions/25653735/freqdist-plot-not-using-the-most-common-words
'''

def plot_freqdist(fd, num = 0, cumulative = False, title = None):
    import pylab
    # Set up parameters
    if num <= 0:
        num = fd.B
    # Get samples and frequencies
    samples, freq, accu = [], [], 0
#     most common words
    for s, f in fd.most_common(num):
#         least common words 
#     for s, f in fd.most_common()[-num:]:
        accu = accu + f if cumulative else f
        samples.append(s)
        freq.append(accu)
    # Create plot
#     pylab.grid(False, color = 'silver')
    if title:
        pylab.title(title)
    pylab.plot(freq, linewidth = 2)
    pylab.xticks(range(len(samples)), samples, rotation = 90)
    pylab.title("The 20 most frequent words and their counts")
    pylab.xlabel('Words')
    pylab.ylabel('Cumulative Counts' if cumulative else 'Counts')
    pylab.show()

In [16]:
plot_freqdist(FreqDist(clear_all_sents), 20)

In [17]:
plot_freqdist(FreqDist(clear_all_sents), 30)

# computing the differences beween different arguments 

The words used to build classifiers includes two parts:
* most common words
* the distinctive words that only appear in one kind of argument

Although some words appear in different kinds of arguments, but the patter is quite different. 

In [18]:
set_premise = set(clear_premise_sents)
set_major_claim_sents = set(clear_major_claim_sents)
set_claim_sents = set(clear_claim_sents)
set_other_arg_sents = set(other_arg_sents)

In [19]:
uniq_premise_words = set_premise - set_major_claim_sents - set_claim_sents - set_other_arg_sents
uniq_major_claim_sents = set_major_claim_sents - set_premise - set_claim_sents - set_other_arg_sents
uniq_claim_sents = set_claim_sents - set_premise - set_major_claim_sents - set_other_arg_sents
uniq_other_arg_sents = set_premise - set_major_claim_sents - set_claim_sents

In [20]:
# print (uniq_claim_sents)
FreqDist(clear_claim_sents)['entering']

1

## save the result 


In [21]:
# clear_premise_sents = clear_split_str(premise_sents)
# clear_major_claim_sents = clear_split_str(major_claim_sents)
# clear_claim_sents = clear_split_str(claim_sents)
# clear_all_claim_sents = clear_split_str(all_claim_sents)
# clear_other_arg_sents = clear_split_str(other_arg_sents)
clear_all_sents = clear_split_str(all_sents)

In [22]:
common_words = {}
common_words['all'] = clear_all_sents
common_words['premise'] = clear_premise_sents
common_words['major_claim'] = clear_major_claim_sents
common_words['claim'] = clear_claim_sents
common_words['all_claim'] = clear_all_claim_sents
common_words['other_arg'] = clear_other_arg_sents

In [23]:
with open('argument_words.pickle', 'wb') as handle:
    pickle.dump(common_words, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# save the 